In [1]:
# canonical import
import pandas as pd

# importing data and dropping na

In [2]:
from google.colab import files
uploaded = files.upload()

Saving reddit_wsb.csv to reddit_wsb.csv


In [3]:
reddit_df = pd.read_csv('reddit_wsb.csv')
reddit_df = reddit_df.dropna()

In [4]:
def include_ticker(string):
    ticker_list = ['GME', 'TSLA']
    for ticker in ticker_list:
        if ticker in string:
            return ticker

In [5]:
! pip install unidecode

     |████████████████████████████████| 245kB 10.1MB/s 


In [6]:
from unidecode import unidecode

In [7]:
def get_unidecode(string):
  return unidecode(string)

In [9]:
reddit_df['body_unidecode']= reddit_df['body'].map(lambda x: get_unidecode(x))
reddit_df['title_unidecode']= reddit_df['title'].map(lambda x: get_unidecode(x))


# Marking the DF

### getting the stock

In [10]:
reddit_df['stock_body'] = reddit_df['body_unidecode'].map(lambda x: include_ticker(x))
reddit_df['stock_title'] = reddit_df['title_unidecode'].map(lambda x: include_ticker(x))

reddit_df

,title,score,id,url,comms_num,created,body,timestamp,body_unidecode,title_unidecode,stock_body,stock_title
2,Exit the system,0,l6uhhn,https://www.reddit.com/r/wallstreetbets/commen...,47,1.611862e+09,The CEO of NASDAQ pushed to halt trading “to g...,2021-01-28 21:30:35,"The CEO of NASDAQ pushed to halt trading ""to g...",Exit the system,GME,None
6,SHORT STOCK DOESN'T HAVE AN EXPIRATION DATE,317,l6uf6d,https://www.reddit.com/r/wallstreetbets/commen...,53,1.611862e+09,Hedgefund whales are spreading disinfo saying ...,2021-01-28 21:26:27,Hedgefund whales are spreading disinfo saying ...,SHORT STOCK DOESN'T HAVE AN EXPIRATION DATE,GME,None
7,THIS IS THE MOMENT,405,l6ub9l,https://www.reddit.com/r/wallstreetbets/commen...,178,1.611862e+09,Life isn't fair. My mother always told me that...,2021-01-28 21:19:31,Life isn't fair. My mother always told me that...,THIS IS THE MOMENT,GME,None
10,"We need to keep this movement going, we all ca...",222,l6uao1,https://www.reddit.com/r/wallstreetbets/commen...,70,1.611862e+09,I believe right now is one of those rare oppo...,2021-01-28 21:18:25,I believe right now is one of those rare oppo...,"We need to keep this movement going, we all ca...",GME,None
12,"Once you're done with GME - $AG and $SLV, the ...",0,l6u9wu,https://www.reddit.com/r/wallstreetbets/commen...,16,1.611861e+09,You guys are champs. GME... who would have tho...,2021-01-28 21:17:10,You guys are champs. GME... who would have tho...,"Once you're done with GME - $AG and $SLV, the ...",GME,GME
...,...,...,...,...,...,...,...,...,...,...,...,...
50689,Weird S&P 500 Correlation to Euro Cup & World Cup,0,o4qwae,https://www.reddit.com/r/wallstreetbets/commen...,14,1.624295e+09,Following is the data and the corresponding S&...,2021-06-21 19:56:54,Following is the data and the corresponding S&...,Weird S&P 500 Correlation to Euro Cup & World Cup,None,None
50690,I $ZIM to the moon,0,o4qu51,https://www.reddit.com/r/wallstreetbets/commen...,10,1.624294e+09,ZIM Integrated Shipping Services should be at ...,2021-06-21 19:52:28,ZIM Integrated Shipping Services should be at ...,I $ZIM to the moon,None,None
50693,Atypical DD for WSB: ATSG for a smooth ascent,29,o4mcu4,https://www.reddit.com/r/wallstreetbets/commen...,28,1.624276e+09,**TLDR: ATSG is a fantastic value at its curre...,2021-06-21 14:48:18,**TLDR: ATSG is a fantastic value at its curre...,Atypical DD for WSB: ATSG for a smooth ascent,None,None
50694,SENS DD,166,o4m5ju,https://www.reddit.com/r/wallstreetbets/commen...,63,1.624275e+09,SENS why to buy\n Skip to end to see summary \...,2021-06-21 14:36:26,SENS why to buy\n Skip to end to see summary \...,SENS DD,GME,None


In [11]:
def body_size(x):
    return len(x.split())

In [12]:
reddit_df['body_size'] = reddit_df['body'].map(lambda x: body_size(x))
reddit_df.reset_index()

,index,title,score,id,url,comms_num,created,body,timestamp,body_unidecode,title_unidecode,stock_body,stock_title,body_size
0,2,Exit the system,0,l6uhhn,https://www.reddit.com/r/wallstreetbets/commen...,47,1.611862e+09,The CEO of NASDAQ pushed to halt trading “to g...,2021-01-28 21:30:35,"The CEO of NASDAQ pushed to halt trading ""to g...",Exit the system,GME,None,226
1,6,SHORT STOCK DOESN'T HAVE AN EXPIRATION DATE,317,l6uf6d,https://www.reddit.com/r/wallstreetbets/commen...,53,1.611862e+09,Hedgefund whales are spreading disinfo saying ...,2021-01-28 21:26:27,Hedgefund whales are spreading disinfo saying ...,SHORT STOCK DOESN'T HAVE AN EXPIRATION DATE,GME,None,116
2,7,THIS IS THE MOMENT,405,l6ub9l,https://www.reddit.com/r/wallstreetbets/commen...,178,1.611862e+09,Life isn't fair. My mother always told me that...,2021-01-28 21:19:31,Life isn't fair. My mother always told me that...,THIS IS THE MOMENT,GME,None,352
3,10,"We need to keep this movement going, we all ca...",222,l6uao1,https://www.reddit.com/r/wallstreetbets/commen...,70,1.611862e+09,I believe right now is one of those rare oppo...,2021-01-28 21:18:25,I believe right now is one of those rare oppo...,"We need to keep this movement going, we all ca...",GME,None,138
4,12,"Once you're done with GME - $AG and $SLV, the ...",0,l6u9wu,https://www.reddit.com/r/wallstreetbets/commen...,16,1.611861e+09,You guys are champs. GME... who would have tho...,2021-01-28 21:17:10,You guys are champs. GME... who would have tho...,"Once you're done with GME - $AG and $SLV, the ...",GME,GME,260
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23591,50689,Weird S&P 500 Correlation to Euro Cup & World Cup,0,o4qwae,https://www.reddit.com/r/wallstreetbets/commen...,14,1.624295e+09,Following is the data and the corresponding S&...,2021-06-21 19:56:54,Following is the data and the corresponding S&...,Weird S&P 500 Correlation to Euro Cup & World Cup,None,None,222
23592,50690,I $ZIM to the moon,0,o4qu51,https://www.reddit.com/r/wallstreetbets/commen...,10,1.624294e+09,ZIM Integrated Shipping Services should be at ...,2021-06-21 19:52:28,ZIM Integrated Shipping Services should be at ...,I $ZIM to the moon,None,None,173
23593,50693,Atypical DD for WSB: ATSG for a smooth ascent,29,o4mcu4,https://www.reddit.com/r/wallstreetbets/commen...,28,1.624276e+09,**TLDR: ATSG is a fantastic value at its curre...,2021-06-21 14:48:18,**TLDR: ATSG is a fantastic value at its curre...,Atypical DD for WSB: ATSG for a smooth ascent,None,None,904
23594,50694,SENS DD,166,o4m5ju,https://www.reddit.com/r/wallstreetbets/commen...,63,1.624275e+09,SENS why to buy\n Skip to end to see summary \...,2021-06-21 14:36:26,SENS why to buy\n Skip to end to see summary \...,SENS DD,GME,None,957


### getting month

### get target data

In [13]:
reddit_df.drop(reddit_df[reddit_df['stock_body'] != "GME"].index, inplace=True)
reddit_df.shape

(6841, 13)

In [14]:
#retirar aquelas células a mais com o sol do demônio
reddit_df = reddit_df[~reddit_df['title'].str.contains("TODAY IS JUDGEMENT DAY")]
reddit_df = reddit_df[~reddit_df['title'].str.contains('BB to the moon!!!')]
reddit_df.shape

(6839, 13)

### getting sentimental

In [15]:
! pip install transformers

     |████████████████████████████████| 2.5MB 7.8MB/s 
     |████████████████████████████████| 3.3MB 44.3MB/s 
     |████████████████████████████████| 901kB 29.0MB/s 


In [16]:
from tensorflow.keras import *

from transformers import pipeline

In [17]:
classifier = pipeline('sentiment-analysis',model="ProsusAI/finbert")

In [19]:
def get_sentimental(x):
    text_chunk = x[0:500]
    print(text_chunk)
    sentimental = classifier(text_chunk)[0]['label']
    print(sentimental)
    return sentimental

In [20]:
title_list = reddit_df['title_unidecode']

In [21]:
body_list = reddit_df['body_unidecode']

In [22]:
title_sentimental = []
for text in title_list:
  x = get_sentimental(text)
  title_sentimental.append(x)

A saída de streaming foi truncada nas últimas 5000 linhas.
It's time to distance ourselves from the memes/noise.
neutral
If you were the hedge funds what would you do?
neutral
Discussion Question: When the squeeze is squoze, how many trading days do you anonymous apes and retards (who are not providing financial advice) think the rapid rise will last? And which day will have the highest intraday spike?
neutral
Citadel HOLD Loss >200%
neutral
Not sure who needs to see this...
neutral
I don't think it's a good idea to buy more GME stocks at this point
neutral
Simple Questions Reveal the Obvious Deceptions
neutral
The Difference between GME and AMC
neutral
Noticed something interesting about the potential Reddit and Discord bots
neutral
Why don't I have any money??
neutral
The idea of buying & holding GME to stick to the "big guys" is bizarre
neutral
AMC Gamma Squeeze
neutral
Robinshit position limits on GME removed
neutral
If your holding now you're just holding the bag for hedge funds
n

In [23]:
reddit_df['title_sentiment_fin'] = title_sentimental

In [24]:
body_sentimental = []
for text in body_list:
  x = get_sentimental(text)
  body_sentimental.append(x)

A saída de streaming foi truncada nas últimas 5000 linhas.

Upfront disclaimer - I've been lurking here for awhile, decided it was time to join and post. I am not an animal. I am a dad of 2, I quit my job on a commodity trade floor 2 years ago (not a trader) after my son was born, and I recently started a business. I've traded stocks for more than 10 years, haven't ever consistently beat the market. I do not offer any type of financial advice, and never plan to. I'm not a teacher, and believe there are stupid questions. Be an adult and do your own work
neutral
My grandma is dying (rest her soul, she's a saint). Next week I have a very long 14 hour drive ahead of me for her to see me, my wife, and the kiddos one last time. A few days later I have the same drive back. I go through long periods of no cell connection, and let's be honest Fidelity's app isn't the greatest for active monitoring of your positions. I'm diamond handing this shit but I'm worried I'll miss the rocket. It's tempti

In [25]:
reddit_df['body_sentiment'] = body_sentimental
reddit_df.to_csv('reddit_with_sentiments.csv')
# reddit_df.drop(reddit_df[reddit_df['title_sentimental'] == "Nan"].index, inplace=True)

In [26]:
# classifier = pipeline('sentiment-analysis')
reddit_df
# classifier(reddit_df.iloc[23594]['body'])

,title,score,id,url,comms_num,created,body,timestamp,body_unidecode,title_unidecode,stock_body,stock_title,body_size,title_sentiment_fin,body_sentiment
2,Exit the system,0,l6uhhn,https://www.reddit.com/r/wallstreetbets/commen...,47,1.611862e+09,The CEO of NASDAQ pushed to halt trading “to g...,2021-01-28 21:30:35,"The CEO of NASDAQ pushed to halt trading ""to g...",Exit the system,GME,None,226,neutral,negative
6,SHORT STOCK DOESN'T HAVE AN EXPIRATION DATE,317,l6uf6d,https://www.reddit.com/r/wallstreetbets/commen...,53,1.611862e+09,Hedgefund whales are spreading disinfo saying ...,2021-01-28 21:26:27,Hedgefund whales are spreading disinfo saying ...,SHORT STOCK DOESN'T HAVE AN EXPIRATION DATE,GME,None,116,neutral,neutral
7,THIS IS THE MOMENT,405,l6ub9l,https://www.reddit.com/r/wallstreetbets/commen...,178,1.611862e+09,Life isn't fair. My mother always told me that...,2021-01-28 21:19:31,Life isn't fair. My mother always told me that...,THIS IS THE MOMENT,GME,None,352,neutral,neutral
10,"We need to keep this movement going, we all ca...",222,l6uao1,https://www.reddit.com/r/wallstreetbets/commen...,70,1.611862e+09,I believe right now is one of those rare oppo...,2021-01-28 21:18:25,I believe right now is one of those rare oppo...,"We need to keep this movement going, we all ca...",GME,None,138,neutral,neutral
12,"Once you're done with GME - $AG and $SLV, the ...",0,l6u9wu,https://www.reddit.com/r/wallstreetbets/commen...,16,1.611861e+09,You guys are champs. GME... who would have tho...,2021-01-28 21:17:10,You guys are champs. GME... who would have tho...,"Once you're done with GME - $AG and $SLV, the ...",GME,GME,260,neutral,neutral
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50568,Marble EFT Report,0,o5bvxd,https://www.reddit.com/r/wallstreetbets/commen...,7,1.624357e+09,# Good evening apes!\n\nSit back and prepare y...,2021-06-22 13:17:48,# Good evening apes!\n\nSit back and prepare y...,Marble EFT Report,GME,None,239,neutral,neutral
50607,"Daily option statistics for WISH, AMC, BB, GME...",501,o542tb,https://www.reddit.com/r/wallstreetbets/commen...,152,1.624335e+09,"Hey guys,\n\nIts time for the daily option sta...",2021-06-22 07:02:45,"Hey guys,\n\nIts time for the daily option sta...","Daily option statistics for WISH, AMC, BB, GME...",GME,GME,131,neutral,negative
50620,Boring Stock to the Moon! Buy Macy's!,0,o52mfx,https://www.reddit.com/r/wallstreetbets/commen...,42,1.624331e+09,Been lurking here and tired of seeing the same...,2021-06-22 05:59:52,Been lurking here and tired of seeing the same...,Boring Stock to the Moon! Buy Macy's!,GME,None,707,neutral,neutral
50671,"$RKT positioned for a textbook ""S word who can...",240,o4vvl4,https://www.reddit.com/r/wallstreetbets/commen...,125,1.624313e+09,"Hello Apes, I'm taking off my fundamentals ha...",2021-06-22 01:05:13,"Hello Apes, I'm taking off my fundamentals ha...","$RKT positioned for a textbook ""S word who can...",GME,None,397,neutral,neutral


In [ ]:
# DO NOT DESTROY THIS CELLL

# " ".join(reddit_df.iloc[23594]['body'].split()[0:427])
# " ".join(reddit_df.iloc[23594]['body'].split()[427:-1])
# print(classifier(" ".join(reddit_df.iloc[23594]['body'].split()[0:427])))
# print(classifier(" ".join(reddit_df.iloc[23594]['body'].split()[-370:-1])))

# " ".join(reddit_df.iloc[23594]['body'].split()[-370:-1])
